In [2]:
import re
import csv

ddl = open('../../sql/ddl/create_table.sql', encoding='utf-8').read()

In [3]:
tables = re.findall(r'CREATE TABLE `(.+?)`\s*\((.+?)\);', ddl, re.DOTALL)

rows = []
for table_name, columns in tables:
    
    schema, table_name = table_name.split(sep=".", maxsplit=-1) # Split [schema, table]
    
    schema = schema[:len(schema)-1] # Clean schema_name
    table_name = table_name[1:] # Clean table_name
    
    for col in columns.split('\n'):
        m = re.match(r'\s*`(.+?)` ([A-Z]+[A-Z0-9_\(\)]*)[^,]*(?:COMMENT \'([^\']+)\')?', col.strip(), re.I)
        print(f"pre-m {m}")
        if m:
            column_name, data_type, comment = m.groups()
            print(f"MMMM {m.groups()}")
            rows.append([table_name, column_name, data_type, comment or ''])

with open('data_dictionary.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Table Name', 'Field Name', 'Data Type', 'Comment'])
    writer.writerows(rows)


pre-m None
pre-m <re.Match object; span=(0, 56), match="`customer_id` INT AUTO_INCREMENT COMMENT 'Mã khác>
MMMM ('customer_id', 'INT', None)
pre-m <re.Match object; span=(0, 61), match="`customer_name` VARCHAR(80) NOT NULL COMMENT 'Tên>
MMMM ('customer_name', 'VARCHAR(80)', None)
pre-m <re.Match object; span=(0, 19), match='`email` VARCHAR(50)'>
MMMM ('email', 'VARCHAR(50)', None)
pre-m None
pre-m None
pre-m <re.Match object; span=(0, 62), match="`manufacturer_id` INT AUTO_INCREMENT COMMENT 'Mã >
MMMM ('manufacturer_id', 'INT', None)
pre-m <re.Match object; span=(0, 67), match="`manufacturer_name` VARCHAR(80) NOT NULL COMMENT >
MMMM ('manufacturer_name', 'VARCHAR(80)', None)
pre-m None
pre-m None
pre-m <re.Match object; span=(0, 63), match="`category_id` INT AUTO_INCREMENT COMMENT 'Mã danh>
MMMM ('category_id', 'INT', None)
pre-m <re.Match object; span=(0, 68), match="`category_name` VARCHAR(80) NOT NULL COMMENT 'Tên>
MMMM ('category_name', 'VARCHAR(80)', None)
pre-m None
pre-m None
pr

In [4]:
tables[1]

('dim`.`manufacturers',
 "\n  `manufacturer_id` INT AUTO_INCREMENT COMMENT 'Mã nhà sản xuất',\n  `manufacturer_name` VARCHAR(80) NOT NULL COMMENT 'Tên nhà sản xuất'\n")